In [1]:
import numpy as np
import tensorflow as tf
from data_util import parse_bj_aq_data, generate_model_data

/home/cvdev/tf/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 设置动态显存占用
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [3]:
bj_aq_data, stations, bj_aq_stations, bj_aq_stations_merged = parse_bj_aq_data()

NaN in PM2.5 is 23459, 6.508615 %
NaN in PM10 is 96175, 26.683406 %
NaN in NO2 is 21720, 6.026135 %
NaN in CO is 46144, 12.802486 %
NaN in O3 is 23732, 6.584358 %
NaN in SO2 is 21664, 6.010598 %
There are 35 air quality stations in Beijing

The stations in Beijing are:
 {'nongzhanguan_aq', 'wanliu_aq', 'dongsihuan_aq', 'gucheng_aq', 'xizhimenbei_aq', 'huairou_aq', 'yanqin_aq', 'tongzhou_aq', 'yongdingmennei_aq', 'yungang_aq', 'fangshan_aq', 'tiantan_aq', 'mentougou_aq', 'wanshouxigong_aq', 'yufa_aq', 'badaling_aq', 'yizhuang_aq', 'daxing_aq', 'fengtaihuayuan_aq', 'zhiwuyuan_aq', 'guanyuan_aq', 'pingchang_aq', 'liulihe_aq', 'pinggu_aq', 'miyun_aq', 'aotizhongxin_aq', 'qianmen_aq', 'dingling_aq', 'dongsi_aq', 'yongledian_aq', 'beibuxinqu_aq', 'shunyi_aq', 'donggaocun_aq', 'nansanhuan_aq', 'miyunshuiku_aq'}


/home/cvdev/air-quality-prediction/data_util.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  bj_aq_station.drop("utc_time", axis=1, inplace=True)
/home/cvdev/air-quality-prediction/data_util.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  bj_aq_station.drop("stationId", axis=1, inplace=True)


In [4]:
X_dataset, Y_dataset = generate_model_data(bj_aq_stations_merged, 1, 120)

In [5]:
print("There are %d examples in the dataset." %len(X_dataset))

There are 10040 examples in the dataset.


In [6]:
# parameters of the model
num_x_hours = 120
num_y_hours = 48
num_features = 210
num_epochs = 1000

data_length = len(X_dataset)

In [7]:
# X = tf.placeholder(dtype=tf.float32, shape=(None, num_x_hours, num_features))
# Y = tf.placeholder(dtype=tf.float32, shape=(None, num_y_hours, num_features))

# not use mini-batch right now
X = tf.placeholder(dtype=tf.float32, shape=(num_x_hours, num_features))
Y = tf.placeholder(dtype=tf.float32, shape=(num_y_hours, num_features))

In [8]:
# # initial values
# initial_W_value = np.random.randn(num_y_hours, num_x_hours) * 1e-10
# initial_b_value = np.random.randn(num_y_hours, num_features) * 1e-10

# # Variables
# W = tf.Variable(initial_W_value, name="W", dtype="float32")
# b = tf.Variable(initial_b_value, name="b", dtype="float32")
W = tf.get_variable(name="W", shape=(num_y_hours, num_x_hours), dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())
b = tf.get_variable(name="b", shape=(num_y_hours, num_features), dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer())

In [9]:
# computation graph
Y_pred = tf.add(tf.matmul(W, X), b)
Loss = tf.reduce_mean(tf.nn.l2_loss(Y_pred - Y))
print(Loss)

# optimizer
Optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(Loss)

Tensor("Mean:0", shape=(), dtype=float32)


In [10]:
losses = []

with tf.device("/gpu:1"):
    with tf.Session(config=config) as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(num_epochs):
            for i in range(data_length):
                _, loss = sess.run([Optimizer, Loss], feed_dict={X:X_dataset[i],Y:Y_dataset[i]})
            if epoch % 100 == 0:
                print(loss)
            losses.append(loss)

21752672.0
21622282.0
21622284.0
21622284.0
21622284.0
21622284.0
21622284.0
21622280.0
21622280.0
21622282.0


In [ ]:
# 这种计算方式对 GPU 的利用效率太低了
# 显存占用 400M，GPU占用 < 1%

In [ ]:
# 预测模型
def predict(parameters, X):
    W, b = parameters
    Y_pred = Y_pred = tf.add(tf.matmul(W, X), b)

In [ ]:
# 将预测结果和实际结果绘制在同一个图形中
def plot_prediction(feature, Y, Y_pred):
    Y_feature = 

In [ ]:
# 划分数据集